<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/NEW_teststrat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pandas matplotlib yfinance

In [4]:
# !curl -L http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O && tar xzvf ta-lib-0.4.0-src.tar.gz
# !cd ta-lib && ./configure --prefix=/usr && make && make install && cd - && pip install ta-lib
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
!pip install conda-package-handling
!wget https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
!cph x ta-lib-0.5.1-py311h9ecbd09_0.conda
!mv ./ta-lib-0.5.1-py311h9ecbd09_0/lib/python3.11/site-packages/talib /usr/local/lib/python3.11/dist-packages/


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4011    0  4011    0     0  23081      0 --:--:-- --:--:-- --:--:-- 23184
100  517k  100  517k    0     0  1179k      0 --:--:-- --:--:-- --:--:-- 5020k
--2025-06-25 10:06:53--  https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
Resolving anaconda.org (anaconda.org)... 104.19.145.37, 104.19.144.37, 2606:4700::6813:9125, ...
Connecting to anaconda.org (anaconda.org)|104.19.145.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5f738b4b78d4370a69f82984/673b62920c609ff6945bb24d?response-content-disposition=attachment%3B%20filename%3D%22ta-lib-0.5.1-py311h9ecbd09_0.conda%22%3B%20filename%2A%3DUTF-8%27%27ta-lib-0.5.1-py311h9ecbd09_0.conda&response-content-type=application%2Foctet-stream&X-Amz-Algorithm=

In [5]:
import pandas as pd
import talib as ta
import yfinance as yf
from datetime import date as date
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt

In [6]:
file_path = '/content/EQUITY_L.xlsx'
fields = ['SYMBOL']

# df = pd.read_excel(file_path)
# print(df.columns.to_list())
tickers = pd.read_excel(file_path, usecols=fields)

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')

unwanted = {'INDO-RE.NS','CDSL.NS','HERIT-RE.NS','ZODIAC.NS','EMAMIPAP.NS','DIGJAMLMTD.NS','GUJRAFFIA.NS','NIRAJISPAT.NS','KALYANI.NS','ARTEMISMED.NS'}
ath_list = [e for e in ath_list if e not in unwanted]

# ath_list

In [7]:
#!rm -rf database
!mkdir database

mkdir: cannot create directory ‘database’: File exists


In [8]:
#!pip install numpy==1.23.0 --force-reinstall
# !pip install pandas-ta
# !pip install -U pandas-ta
# import pandas_ta as pta

In [9]:
startdate = date.today() - relativedelta(years=4)
enddate = date.today() + relativedelta(days=1)

for company in ath_list:
  try:
    df = yf.download(company, start= startdate, end= enddate)
    df.to_csv(f"database/{company.split('.NS')[0]}.csv")
  except:
    pass


Streaming output truncated to the last 5000 lines.
  df = yf.download(company, start= startdate, end= enddate)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-9-1270418567.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(company, start= startdate, end= enddate)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-9-1270418567.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(company, start= startdate, end= enddate)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-9-1270418567.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(company, start= startdate, end= enddate)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-9-1270418567.py:6: FutureWarning: YF.download() has changed argume

In [13]:
import os
datafiles = os.listdir('database')

for filename in datafiles:
  df = pd.read_csv(f"database/{filename}")
  try:
    result = ta.CDLENGULFING(df['Open'], df['High'], df['Low'], df['Close'] )
    df['EMA200'] = ta.ema(df['Close'], timeperiod=200)
    df['INTRA_perc'] = ((df['Close'] - df['Close'].shift(1))/df['Close'].shift(1))*100
    df['TOD_perc'] = ((df['Close'] - df['Open'])/df['Open'])*100
    prev_close = df.groupby('Date').Close.tail(1).shift(1).tail(1).values[0]
    last = result.tail(1).values[0]
    last_10 = result.tail(12)

    if df['EMA200'].tail(1).values[0] < df['Close'].tail(1).values[0]:
      if last > 0:
        print(f"{filename.split('.csv')[0]} was triggered , Bullish , EMA200 = {df['EMA200'].tail(1).values[0]} , Last Close = {df['Close'].tail(1).values[0]}, Prev day Close = {prev_close},  \
         INTRA % Change = {df['INTRA_perc'].tail(1).values[0]} , Today % Change = {df['TOD_perc'].tail(1).values[0]} ")
      #   print(last_10)
      elif last < 0:
        print(f"{filename.split('.csv')[0]} was triggered , Bearish")
  except:
    pass

#df_1d = df_1d.swaplevel(0, 1, 1).sort_index(axis=1)

In [11]:
result = ta.CDLENGULFING(df_1d[:,'Open'], df_1d[:,'High'], df_1d[:,'Low'], df_1d[:,'Close'] )

In [12]:
!pip index versions pandas-ta

ERROR: No matching distribution found for pandas-ta
